In [46]:
import os
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [2]:
os.chdir('..')

In [3]:
pwd

'/teamspace/studios/this_studio/CreditRiskAI'

In [4]:
spark = SparkSession.builder.appName("EDA").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/18 06:04:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
data_label_path = "data/raw/train_labels.csv"
train_data_path = "data/raw/train_data.csv"

In [6]:
label_df = spark.read.option("header", "true").csv(data_label_path)

In [7]:
label_df.show(10)

+--------------------+------+
|         customer_ID|target|
+--------------------+------+
|0000099d6bd597052...|     0|
|00000fd6641609c6e...|     0|
|00001b22f846c82c5...|     0|
|000041bdba6ecadd8...|     0|
|00007889e4fcd2614...|     0|
|000084e5023181993...|     0|
|000098081fde4fd64...|     0|
|0000d17a1447b25a0...|     0|
|0000f99513770170a...|     1|
|00013181a0c5fc8f1...|     1|
+--------------------+------+
only showing top 10 rows



In [8]:
label_df.printSchema()

root
 |-- customer_ID: string (nullable = true)
 |-- target: string (nullable = true)



In [10]:
label_df.count()

458913

In [9]:
label_df.filter(label_df.target.isNull()).show()


+-----------+------+
|customer_ID|target|
+-----------+------+
+-----------+------+



In [11]:
label_df.filter(label_df.customer_ID.isNull()).show()

+-----------+------+
|customer_ID|target|
+-----------+------+
+-----------+------+



In [12]:
clean_label_df = label_df.withColumn("target", F.col("target").cast('float'))
clean_label_df.printSchema()

root
 |-- customer_ID: string (nullable = true)
 |-- target: float (nullable = true)



In [15]:
output_path = "data/processed/cleaned_train_label"
df_single_partition = clean_label_df.coalesce(1)
df_single_partition.write.parquet(path= output_path, compression= "gzip")

In [16]:
train_dataframe = spark.read.option("header", "True").csv(path = train_data_path, inferSchema = True,)
train_dataframe.show(5)

24/03/18 06:17:41 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+----------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+------------------+----+----+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+----+------------------+------------------+------------------+------------------+------------------+--------------------+------------------+------------------+------------------+------------------+----+------------------+------------------+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+----+----+------------------+------------------

In [17]:
columns_names = train_dataframe.columns
len(columns_names)

190

In [32]:
train_dataframe.printSchema()

root
 |-- customer_ID: string (nullable = true)
 |-- S_2: date (nullable = true)
 |-- P_2: double (nullable = true)
 |-- D_39: double (nullable = true)
 |-- B_1: double (nullable = true)
 |-- B_2: double (nullable = true)
 |-- R_1: double (nullable = true)
 |-- S_3: double (nullable = true)
 |-- D_41: double (nullable = true)
 |-- B_3: double (nullable = true)
 |-- D_42: double (nullable = true)
 |-- D_43: double (nullable = true)
 |-- D_44: double (nullable = true)
 |-- B_4: double (nullable = true)
 |-- D_45: double (nullable = true)
 |-- B_5: double (nullable = true)
 |-- R_2: double (nullable = true)
 |-- D_46: double (nullable = true)
 |-- D_47: double (nullable = true)
 |-- D_48: double (nullable = true)
 |-- D_49: double (nullable = true)
 |-- B_6: double (nullable = true)
 |-- B_7: double (nullable = true)
 |-- B_8: double (nullable = true)
 |-- D_50: double (nullable = true)
 |-- D_51: double (nullable = true)
 |-- B_9: double (nullable = true)
 |-- R_3: double (nullable = tru

In [19]:
categorical_features = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
len(categorical_features)

11

In [20]:
numerical_features  = [name for name in columns_names if name not in categorical_features]
len(numerical_features)

179

Divide the columns into ten batch

In [25]:
batch_cols = [train_dataframe.select(columns_names[num:num + 20]) for num in range(0, 100, 20)]

In [30]:
current_batch = batch_cols[0]

In [32]:
current_batch.show(5)

+--------------------+----------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+------------------+----+----+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|         customer_ID|       S_2|               P_2|              D_39|               B_1|               B_2|               R_1|               S_3|                D_41|               B_3|D_42|D_43|               D_44|               B_4|              D_45|               B_5|               R_2|              D_46|              D_47|              D_48|
+--------------------+----------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+------------------+----+----+-------------------+------------------+------------------+------------------+------------------+-----

In [34]:
current_batch.printSchema()

root
 |-- customer_ID: string (nullable = true)
 |-- S_2: date (nullable = true)
 |-- P_2: double (nullable = true)
 |-- D_39: double (nullable = true)
 |-- B_1: double (nullable = true)
 |-- B_2: double (nullable = true)
 |-- R_1: double (nullable = true)
 |-- S_3: double (nullable = true)
 |-- D_41: double (nullable = true)
 |-- B_3: double (nullable = true)
 |-- D_42: double (nullable = true)
 |-- D_43: double (nullable = true)
 |-- D_44: double (nullable = true)
 |-- B_4: double (nullable = true)
 |-- D_45: double (nullable = true)
 |-- B_5: double (nullable = true)
 |-- R_2: double (nullable = true)
 |-- D_46: double (nullable = true)
 |-- D_47: double (nullable = true)
 |-- D_48: double (nullable = true)



In [36]:
for col_name  in current_batch.columns[2:]:
    current_batch = current_batch.withColumn(col_name, F.col(col_name).cast("float"))

In [38]:
current_batch.printSchema()

root
 |-- customer_ID: string (nullable = true)
 |-- S_2: date (nullable = true)
 |-- P_2: double (nullable = true)
 |-- D_39: float (nullable = true)
 |-- B_1: float (nullable = true)
 |-- B_2: float (nullable = true)
 |-- R_1: float (nullable = true)
 |-- S_3: float (nullable = true)
 |-- D_41: float (nullable = true)
 |-- B_3: float (nullable = true)
 |-- D_42: float (nullable = true)
 |-- D_43: float (nullable = true)
 |-- D_44: float (nullable = true)
 |-- B_4: float (nullable = true)
 |-- D_45: float (nullable = true)
 |-- B_5: float (nullable = true)
 |-- R_2: float (nullable = true)
 |-- D_46: float (nullable = true)
 |-- D_47: float (nullable = true)
 |-- D_48: float (nullable = true)



In [40]:
current_batch.show(10)

+--------------------+----------+------------------+------------+------------+---------+------------+-----------+------------+------------+----+----+------------+------------+----------+-----------+------------+----------+----------+----------+
|         customer_ID|       S_2|               P_2|        D_39|         B_1|      B_2|         R_1|        S_3|        D_41|         B_3|D_42|D_43|        D_44|         B_4|      D_45|        B_5|         R_2|      D_46|      D_47|      D_48|
+--------------------+----------+------------------+------------+------------+---------+------------+-----------+------------+------------+----+----+------------+------------+----------+-----------+------------+----------+----------+----------+
|0000099d6bd597052...|2017-03-09|0.9384687191272548| 0.001733339| 0.008724451|1.0068382| 0.009227722| 0.12403516| 0.008771132|0.0047092405|NULL|NULL|6.3013483E-4| 0.080986336| 0.7089063| 0.17060024|0.0062040314|0.35858658|0.52535105|0.25573608|
|0000099d6bd597052..

In [66]:
current_batch.describe("D_42").show()

+-------+-------------------+
|summary|               D_42|
+-------+-------------------+
|  count|             791314|
|   mean|0.18497419533974807|
| stddev|0.22818457987906962|
|    min|      -4.5433026E-4|
|    max|          4.1911187|
+-------+-------------------+

